Using `SmugPyter` Batch Scripts
===============

![](jupysmug.png)

### Maintaining Local SmugMirror Directories

`SmugPyter` maintains local directories of SmugMug metadata. The directories
mirror your online folders and albums and contain:

1. Metadata in simple CSV TAB delimited text files.

2. Sample images.

This information is used to set:

1. Print size keywords.

2. Geotag keywords.

3. Dominant color keywords.

The SmugMirror is primarily an input to other programs including version control systems like `Git` that
can track changes in text files.

Keeping local directories current is simple matter of running a few batch scripts.

### Clearing Changes Files.

Keyword changes are staged in changes files. Changes files follow this naming convention.


Before updating local files it's a good idea to empty existing changes files.

In [6]:
# Empty all CSV changes files: assumes (smugpyter) location is on sys.path.
import smugpyter

smug = smugpyter.SmugPyter()
print(smug.reset_all_changes_files('c:\SmugMirror'))

(57, 57)


Reseting changes files is also available via the batch script.

     ResetAllChangesFiles.bat

### Refreshing Manifest Files

The leaf directories in SmugMirror contain CSV TAB delimited text files that list images in SmugMug albums.
Manifest files follow this naming convention:



Manifest files are keys inputs to functions in `SmugPyter` and its subclasses. Here is how you keep them updated.

In [2]:
# Refresh album CSV manifest files: assumes (smugpyter) location is on sys.path.
import smugpyter

smug = smugpyter.SmugPyter()
smug.download_smugmug_mirror(func_album=smugmug.write_album_manifest)

visiting album GreatandGreaterForebearers
visiting album MinnieRaver
visiting album Grandparents
visiting album MyKids
visiting album TheWayWeWere
visiting album FromHazelsAlbums
visiting album InlawsOutlawsandFriends
visiting album MyWifesFamily
visiting album HelenHamilton
visiting album Video
visiting album IdahoInstants
visiting album InandAroundOttawa
visiting album MontanaNowandThen
visiting album IndianaImages
visiting album Minnesota
visiting album NewMexicoMontage
visiting album MissouriMoments
visiting album KingstonOntario
visiting album CaliforniaCaptures
visiting album Iran1960s
visiting album Ghana1970s
visiting album BeirutLebanon1960s
visiting album DivingatBellairsBarbadosBW
visiting album Weekenders
visiting album WesternRoadTrip2015
visiting album NorthbyNorthwest
visiting album TetonsYellowstone2013
visiting album ArizonaToodling
visiting album AlongtheYukonRiver
visiting album VirginiaFall2010
visiting album Chicago2007
visiting album NewYork2005
visiting album Ban

Refreshing manifest files is also available via the batch script.

     RefreshManifests.bat

### Refreshing Real Dates

SmugMug images are tagged with many dates. The dates easily extracted via the SmugMug 2.0 API are 
typically upload and other system dates. Usually we want EXIF/IPTC dates. To get these dates it's
currently necessary to issue a web API request for every single image. This is time consuming and tedious. 
Fortunately, image real dates are very stable. Once downloaded they seldom change. Hence, real date 
refreshing checks downloaded images for dates before issuing API requests. You can force reloads by
editing manifest and real date files and deleting images before running the following code snippet.

In [3]:
# Refresh album CSV real date files: assumes (smugpyter) is on sys.path.

# WARNING: it takes a long time to run this script for the
# first time. The dates we are looking for are buried in the
# original EXIF/IPTC image values and there appears to be no fast
# way to extract these dates using the SmugMug API 2.0.
# The "fast" API dates are upload and change dates which
# are not image EXIF dates. You have to ping the metadata 
# of every single image in all albums to extract "real dates."
# The good news is that these dates are fairly stable and 
# simple caching of previously fetched dates works well
# for typical incremental updates.

import smugpyter

smug = smugpyter.SmugPyter(yammer=True)
smug.download_smugmug_mirror(func_album=smug.write_album_real_dates)

visiting album GreatandGreaterForebearers
visiting album MinnieRaver
visiting album Grandparents
visiting album MyKids
visiting album TheWayWeWere
visiting album FromHazelsAlbums
visiting album InlawsOutlawsandFriends
visiting album MyWifesFamily
visiting album HelenHamilton
visiting album Video
visiting album IdahoInstants
visiting album InandAroundOttawa
visiting album MontanaNowandThen
visiting album IndianaImages
visiting album Minnesota
visiting album NewMexicoMontage
visiting album MissouriMoments
visiting album KingstonOntario
visiting album CaliforniaCaptures
visiting album Iran1960s
visiting album Ghana1970s
visiting album BeirutLebanon1960s
visiting album DivingatBellairsBarbadosBW
visiting album Weekenders
visiting album WesternRoadTrip2015
visiting album NorthbyNorthwest
visiting album TetonsYellowstone2013
visiting album ArizonaToodling
visiting album AlongtheYukonRiver
visiting album VirginiaFall2010
visiting album Chicago2007
visiting album NewYork2005
visiting album Ban

Refreshing real dates is also available via the batch script.

    RefreshRealdates.bat

### Geotagging and Reverse Geocoding Images

Images with nonzero latitude and longitude can be geotagged and reverse geocoded, 
(keywords are place names), with the following.

In [1]:
# Update CSV geotagged keywords changes files. Run after 
# refreshing manifest files: assumes (smugpyter, geotagkeys) is on sys.path.
# After running this script apply the changes with (ChangeKeywords.bat)
# and then rerun (RefreshManifests.bat).

import geotagkeys

gk = geotagkeys.GeotagKeys()
root = 'c:\SmugMirror'

# update changes files
# gk.yammer = True
print(gk.set_all_reverse_geocodes(root))

(4259, 0)


Updating geotagged keywords is also available via the batch script.

    UpdateGeotagKeywords.bat

After setting local changes files the keywords can be pushed to SmugMug with:

In [3]:
# Scan CSV changes files in local directories and apply keyword changes.
# Run after refreshing manifest files and updating keyword changes files: 
# assumes (smugpyter) is on sys.path
import smugpyter

smug = smugpyter.SmugPyter()
print(smug.update_all_keyword_changes('c:\SmugMirror'))

(0, 0)


Pushing keyword changes to SmugMug can be done with:

    ChangeKeywords.bat

After applying geotagging keywords refresh manifest files again.

### Computing Print Size and Dominant Color Keys

Print size and dominant color keys are computed with:

In [5]:
# Update CSV print size and color keywords changes files.
# Run after refreshing manifest files and updating
# sample images: assumes (smugpyter, printkeys, 
# colorkeys) are on sys.path.

import printkeys
import colorkeys

pk = printkeys.PrintKeys()
ck = colorkeys.ColorKeys()
root = 'c:\SmugMirror'

# update changes files - order matters
#pk.yammer = True
#pk.merge_changes = True
print(pk.update_all_size_keyword_changes(root))

#ck.yammer = True
ck.merge_changes = True
print(ck.update_all_color_keyword_changes(root))

# change online keywords - same as (ChangeKeywords.bat)
print(pk.update_all_keyword_changes(root))

(4259, 0)
cannot compute color key -> c:\SmugMirror\Other\utilimages\\cNF63Lh-1e-blu-diamond.png
cannot compute color key -> c:\SmugMirror\People\Grandparents\\2fPXg4d-o-frank-baker-portrait-[4445383].jpg
cannot compute color key -> c:\SmugMirror\People\TheWayWeWere\\2fPXg4d-o-frank-baker-portrait-[4445383].jpg
cannot compute color key -> c:\SmugMirror\Places\Overseas\DivingatBellairsBarbadosBW\\XTkjwT7-2q-squid-near-anchor-rope-[6225543].jpg
cannot compute color key -> c:\SmugMirror\Themes\Manipulations\LogosScreenshotsCovers\\VwTq2Zj-2a-jodlittlecube2.png
(4259, 0)
(0, 0)


Finally, print size and color keywords can be set with:

     UpdateSizeColorKeywords.bat

## Next on the Agenda!

The local directories maintained by `SmugPyter` are mainly inputs to other programs.

Remember, always [Analyze the Data not the Drivel](https://analyzethedatanotthedrivel.org/).

John Baker, Meridian Idaho